In [ ]:
! pip install torch
! pip install evalplus --upgrade

In [6]:
basic_prompt = "You task is to complete code. Code should be inserted at the end of the file. Don't generate the whole document and don't add any explanations after the code completion ."
oneshot_prompt = """You task is to complete code. Don't generate the whole document and don't add any explanations after the code completion .
For example if your input is:
```import java.util.Arrays;

public class ArraySum {

    /**
     * Function to calculate the sum of all elements in an array.
     * @param arr The input array of integers.
     * @return The sum of all elements in the array.
     *
     * > sumArray([1, 2, 3])
     * 6
     * > sumArray([2, 3, 4, 2, 6, 4])
     * 21
     */
    public static int sumArray(int[] arr) {
```
You should output:
```
        int sum = 0;
        for (int num : arr) {
            sum += num;
        }
        return sum;
    }
}
```
It is very important that you do not add any explanations after this output.
"""


In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def deepseek(prompt):
    tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True, torch_dtype=torch.bfloat16).cuda()
    messages=[
        { 'role': 'user', 'content': prompt}
    ]
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)
    
    outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, top_k=50, top_p=0.95, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
    res = tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    print(res)
    return res


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


def test_duplicate(arraynums):
    if len(arraynums) == len(set(arraynums)):
        return False
    else:
        return True


'def test_duplicate(arraynums):\n    if len(arraynums) == len(set(arraynums)):\n        return False\n    else:\n        return True\n'

In [ ]:
from evalplus.data import get_mbpp_plus, write_jsonl

samples1 = [
    dict(task_id=task_id, solution=deepseek(oneshot_prompt + problem["prompt"]))
    for task_id, problem in list(get_mbpp_plus().items())[:200]
]


In [ ]:
from evalplus.data import get_mbpp_plus, write_jsonl

samples2 = [
    dict(task_id=task_id, solution=deepseek(oneshot_prompt + problem["prompt"]))
    for task_id, problem in list(get_mbpp_plus().items())[200:]
]


In [ ]:
write_jsonl("samples.jsonl", samples1 + samples2)